In [ ]:
pip  install opencv-python

In [1]:
#Download and upload the test.png file in your local Colab directory
from google.colab import files
import urllib.request

url = 'https://istitutocomprensivo1asti.edu.it/allegati/news/295-2.jpg'
file_name = 'test.png'

urllib.request.urlretrieve(url, file_name)

files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import cv2
import numpy as np

In [3]:
def encode(image_path, output_path, qp):
    # Read the input image
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Convert the image to YCbCr color space
    img_ycbcr = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

    # Split the YCbCr image into Y, Cb, and Cr components
    y, cb, cr = cv2.split(img_ycbcr)

    # Apply Discrete Cosine Transform (DCT) to each component
    y_dct = cv2.dct(np.float32(y))
    cb_dct = cv2.dct(np.float32(cb))
    cr_dct = cv2.dct(np.float32(cr))

    # Quantization
    y_quantized = np.round(y_dct / (qp * np.ones_like(y_dct)))
    cb_quantized = np.round(cb_dct / (qp * np.ones_like(cb_dct)))
    cr_quantized = np.round(cr_dct / (qp * np.ones_like(cr_dct)))

    # Save the quantized data to a binary file
    with open(output_path, 'wb') as file:
        np.save(file, y_quantized)
        np.save(file, cb_quantized)
        np.save(file, cr_quantized)

In [4]:
def decode(input_path, output_path, qp):
    # Load the quantized data from the binary file
    with open(input_path, 'rb') as file:
        y_quantized = np.load(file)
        cb_quantized = np.load(file)
        cr_quantized = np.load(file)

    # Dequantization
    y_dct = y_quantized * (qp * np.ones_like(y_quantized))
    cb_dct = cb_quantized * (qp * np.ones_like(cb_quantized))
    cr_dct = cr_quantized * (qp * np.ones_like(cr_quantized))

    # Apply Inverse Discrete Cosine Transform (IDCT)
    y_reconstructed = cv2.idct(np.float32(y_dct))
    cb_reconstructed = cv2.idct(np.float32(cb_dct))
    cr_reconstructed = cv2.idct(np.float32(cr_dct))

    # Merge the Y, Cb, Cr components
    img_reconstructed_ycbcr = cv2.merge([y_reconstructed, cb_reconstructed, cr_reconstructed])

    # Convert back to BGR color space
    img_reconstructed = cv2.cvtColor(img_reconstructed_ycbcr, cv2.COLOR_YCrCb2BGR)

    # Save the reconstructed image
    cv2.imwrite(output_path, img_reconstructed)

In [5]:
def calculate_psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

In [6]:
# Clarify the path of the files
image_path = '/content/test.png'
compressed_path = '/content/test.bin'
reconstructed_path = '/content/rec.png'

In [7]:
# Adjust the compression rate (QP)
compression_rate = 10

In [8]:
# Encoding
encode(image_path, compressed_path, compression_rate)

In [9]:
# Decoding
decode(compressed_path, reconstructed_path, compression_rate)

In [10]:
# Calculate PSNR
original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
reconstructed_image = cv2.imread(reconstructed_path, cv2.IMREAD_COLOR)

In [11]:
# Show the PSNR
psnr = calculate_psnr(original_image, reconstructed_image)
print(f'Peak Signal-to-Noise Ratio (PSNR): {psnr} dB')

Peak Signal-to-Noise Ratio (PSNR): 28.17096460093576 dB
